In [1]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
import joblib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tkinter import ttk


class DataCleanerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Очистка данных, регрессия и генерация отчета")
        self.root.geometry("600x400")
        self.root.configure(bg="#AFEEEE")  # Цвет фона окна

        # Переменные для хранения путей к файлам
        self.file_path = None
        self.save_path = None

        # Настройка стиля для кнопок
        style = ttk.Style()
        style.theme_use("default")
        style.configure(
            "RoundedButton.TButton",
            background="#FFFFFF",
            foreground="black",
            font=("Roboto Flex", 14),
            borderwidth=0,
            relief="flat",
            padding=10,
            anchor="center",
            borderradius=80  # Радиус скругления углов
        )

        # Элементы GUI
        label1 = tk.Label(
            root,
            text="Выберите Excel или CSV файл:",
            bg="#AFEEEE",
            fg="black",
            font=("Roboto Flex", 14)
        )
        label1.pack(pady=(30, 5))

        self.select_button = ttk.Button(
            root,
            text="Выбрать файл",
            style="RoundedButton.TButton",
            command=self.load_file
        )
        self.select_button.pack(pady=5)

        label2 = tk.Label(
            root,
            text="Выберите место для сохранения отчета и модели:",
            bg="#AFEEEE",
            fg="black",
            font=("Roboto Flex", 14)
        )
        label2.pack(pady=(20, 5))

        self.save_button = ttk.Button(
            root,
            text="Выбрать папку",
            style="RoundedButton.TButton",
            command=self.select_save_location
        )
        self.save_button.pack(pady=5)

        self.process_button = ttk.Button(
            root,
            text="Выгрузить отчет",
            style="RoundedButton.TButton",
            command=self.process_data
        )
        self.process_button.pack(pady=(20, 10))

    def load_file(self):
        """Открывает диалоговое окно для выбора файла."""
        self.file_path = filedialog.askopenfilename(
            filetypes=[("Excel files", "*.xlsx *.xls"), ("CSV files", "*.csv")]
        )
        if self.file_path:
            messagebox.showinfo("Файл выбран", f"Выбран файл: {self.file_path}")

    def select_save_location(self):
        """Открывает диалоговое окно для выбора места сохранения."""
        self.save_path = filedialog.askdirectory()
        if self.save_path:
            messagebox.showinfo("Папка выбрана", f"Результаты будут сохранены в: {self.save_path}")

    def process_data(self):
        """Основная функция для обработки данных, построения регрессии и генерации отчета."""
        if not self.file_path:
            messagebox.showerror("Ошибка", "Сначала выберите файл!")
            return

        if not self.save_path:
            messagebox.showerror("Ошибка", "Сначала выберите место для сохранения!")
            return

        try:
            # Загрузка данных
            if self.file_path.endswith(('.xlsx', '.xls')):
                df = pd.read_excel(self.file_path)
            elif self.file_path.endswith('.csv'):
                df = pd.read_csv(self.file_path)
            else:
                raise ValueError("Неподдерживаемый формат файла!")

            # Очистка данных от NaN
            df_cleaned = df.dropna(how='all')  # Удаляем строки, где все значения NaN

            # Проверка наличия числовых столбцов для регрессии
            numeric_columns = df_cleaned.select_dtypes(include=['number']).columns
            if len(numeric_columns) < 2:
                raise ValueError("Недостаточно числовых столбцов для построения регрессии!")

            # Разделение на признаки (X) и целевую переменную (y)
            X = df_cleaned[numeric_columns[:-1]]
            y = df_cleaned[numeric_columns[-1]]

            # Построение модели линейной регрессии
            model = LinearRegression()
            model.fit(X, y)

            # Сохранение модели
            model_name = os.path.basename(self.file_path).split('.')[0] + "_regression_model.pkl"
            model_path = os.path.join(self.save_path, model_name)
            joblib.dump(model, model_path)

            # Генерация отчета
            profile = ProfileReport(df_cleaned, title="Отчет по данным", explorative=True)

            # Сохранение отчета
            report_name = os.path.basename(self.file_path).split('.')[0] + "_report.html"
            report_path = os.path.join(self.save_path, report_name)
            profile.to_file(report_path)

            # Генерация графика регрессии и сохранение в PDF
            pdf_name = os.path.basename(self.file_path).split('.')[0] + "_regression.pdf"
            pdf_path = os.path.join(self.save_path, pdf_name)

            with PdfPages(pdf_path) as pdf:
                for i, feature in enumerate(X.columns):
                    plt.figure(figsize=(8, 6))
                    plt.scatter(X[feature], y, color='blue', label='Данные')
                    plt.plot(X[feature], model.predict(X), color='red', label='Регрессия')
                    plt.xlabel(feature)
                    plt.ylabel(numeric_columns[-1])
                    plt.title(f"Регрессия для {feature}")
                    plt.legend()
                    pdf.savefig()  # Сохранение страницы в PDF
                    plt.close()

            messagebox.showinfo("Успех",
                                f"Отчет успешно создан и сохранен в:\n{report_path}\n\n"
                                f"Модель регрессии сохранена в:\n{model_path}\n\n"
                                f"Графики регрессии сохранены в:\n{pdf_path}")

        except Exception as e:
            messagebox.showerror("Ошибка", f"Произошла ошибка: {str(e)}")


if __name__ == "__main__":
    root = tk.Tk()
    app = DataCleanerApp(root)
    root.mainloop()